In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install comet_ml

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV

In [ ]:
import comet_ml
from comet_ml import Experiment

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
df

In [ ]:
df.isna().sum().sum()

In [ ]:
df.info()

In [ ]:
df["target"].value_counts()

In [ ]:
df.describe()

In [ ]:
df["f_27"].nunique()

In [ ]:
df["f_27"].value_counts()

In [ ]:
df["f_27"].describe()

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

In [ ]:
X = df.drop(["id", "f_27", "target"], axis=1)

In [ ]:
y = df.target.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [ ]:
parameters = {
        'gamma':[2, 5],
        #'max_depth':[5, 10],
        'n_estimators':[200]
        }

xgbst = xgb.XGBClassifier(learning_rate=1, objective='binary:logistic',
                    silent=True, nthread=1)

kfold = KFold(n_splits=2)


In [ ]:
clf = GridSearchCV(
  xgbst,
  param_grid=parameters,
  cv=kfold,
  scoring='roc_auc',
  refit=True,
  verbose=3,
  n_jobs=-1
)

In [ ]:
model = clf.fit(X_train, y_train)

In [ ]:
for i in range(len(clf.cv_results_['params'])):
    exp = Experiment(api_key="FLIoUDWUoHJRZvyHV3XMXGoGT", project_name="Kaggle_Tabluar_Data_Challenge_2022")
    for k,v in clf.cv_results_.items():
        if k == "params":
            exp.log_parameters(v[i])
        else:
            exp.log_metric(k,v[i])

In [ ]:
pred = model.predict(X_test)
print('Best AUC Score: {}'.format(model.best_score_))
print(confusion_matrix(y_test,pred))

In [ ]:
print(confusion_matrix(y_test,pred))

In [ ]:
exp.end()

In [ ]:
df_sub = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')

In [ ]:
df_sub = df_sub.drop(["id", "f_27"], axis=1)

In [ ]:
df_sub

In [ ]:
preds = model.predict(df_sub)

In [ ]:
sub = pd.DataFrame(np.column_stack((list(df_sub['id']), preds)), columns=["id", "target"])

In [ ]:
sub

In [ ]:
sub.to_csv("submission.csv", index=False)